In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('E:\\Downloads\\train_redux2.csv', dtype='int8')
y = pd.read_csv('E:\\Downloads\\y2.csv', sep=';', usecols=[1], dtype='int8').values.ravel()

print(df.shape, y.shape)

(26495, 8065) (26495,)


In [12]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from keras.layers import LSTM, GRU, Bidirectional
from keras.optimizers import RMSprop
import h5py

s1 = StratifiedShuffleSplit(n_splits=1, test_size=0.20)
for t1_index, val_index in s1.split(df, y):
    s1_x, s1_y = pd.DataFrame(df.iloc[t1_index]), y.iloc[t1_index].values.ravel()
    valid_x, valid_y = df.iloc[val_index], y.iloc[val_index].values.ravel()
 
#cv = StratifiedKFold(n_splits=4, shuffle=True)
cv = StratifiedShuffleSplit(n_splits=1, test_size=0.25)
for train_index, test_index in cv.split(s1_x, s1_y):
    train_x, train_y = pd.DataFrame(s1_x.iloc[train_index]), s1_y[train_index]
    test_x, test_y = pd.DataFrame(s1_x.iloc[test_index]), s1_y[test_index]
    #("batch", "channels", "height", "width")
    
    imput_shape = (3, train_x.shape[1], 1)

    callbacks = [EarlyStopping(monitor='val_loss', patience=100, verbose=1)]
    #checkpointer = ModelCheckpoint(filepath="DanQ-JASPAR_bestmodel.hdf5", verbose=1, save_best_only=True)

    model = Sequential()
    model.add(Conv2D(32, (1, 9), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(1, 2)))

    model.add(Conv2D(32, (1, 9)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(1, 2)))

    model.add(Conv2D(64, (1, 9)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(1, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=100, nb_epoch=32, shuffle=True, \
              validation_data=(X_test, y_test), callbacks=callbacks)

    valid_preds = model.predict_proba(X_test, verbose=0)
    valid_preds = valid_preds[:, 1]
    roc = roc_auc_score(y_test, valid_preds)
    print("ROC:", roc)


C:\Users\Paul\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:15: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
C:\Users\Paul\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:15: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, input_shape=(None, 102..., units=512)`
C:\Users\Paul\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:16: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
C:\Users\Paul\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:16: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, input_shape=(None, 102..., units=512)`
C:\Users\Paul\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:27: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(1000, 4)

compiling model


C:\Users\Paul\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:39: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=65536, units=925)`
C:\Users\Paul\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", input_dim=925, units=919)`
C:\Users\Paul\Anaconda3\envs\tensorflow\lib\site-packages\keras\models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


ValueError: Error when checking input: expected conv1d_4_input to have 3 dimensions, but got array with shape (18546, 8065)

In [46]:
    preds = model.predict_proba(X_test, verbose=0)[:, 1]
    submission = pd.DataFrame(preds, index=ids, columns=['target'])
    submission.to_csv('Keras_BTB.csv')    

NameError: name 'model' is not defined